# Geocoding apartment addresses using the geoadmin API

## Libraries and settings

In [1]:
# Libraries
import os
import requests
import json
import urllib
import fnmatch
import folium
import pandas as pd
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# python: strings zusammenlegen:
'cat'+' '+'dog'

'cat dog'

In [3]:
#get number of rows an columns
# print(df.shape)

#adresse zusammenbauen
# df[''] = df['street_name']+ df['house_number']

## Geocoding a single address

### Define base url for address search

In [4]:
# Define base url for address search
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Set up search parameters: address, origins and type
parameters = {"searchText": "8400 Winterthur, Theaterstrasse 17",
              "origins": "address",
              "type": "locations",
             }

# Urllib.parse.urlencode turns parameters into url
# print(f"{base_url}{urllib.parse.urlencode(parameters)}")

##### Aufgabe 1c

In [5]:
# Define base url for address search
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Set up search parameters: address, origins and type
parameters = {"searchText": "8887 Mels, Täliweg 28",
              "origins": "address",
              "type": "locations",
             }

r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

# Get data in json-format
data = json.loads(r.content)
data

# Take only the first server response, convert to data frame with relevant infos
df = pd.DataFrame.from_dict(list(data.values())[0][0], orient='columns')
df.iloc[[1,4,5,6,11,12],:1]
df

,attrs,id,weight
detail,taeliweg 28 8887 mels 3293 mels ch sg,558838,4
featureId,3149488_0,558838,4
geom_quadindex,030132220233010323320,558838,4
geom_st_box2d,"BOX(750782.1320000002 212080.5929999986,750782...",558838,4
label,Täliweg 28 <b>8887 Mels</b>,558838,4
lat,47.042603,558838,4
lon,9.422916,558838,4
num,28,558838,4
objectclass,,558838,4
origin,address,558838,4


### Server request & response

In [6]:
# featureid zeigt jedes gebäude in der Schweiz (jedes Gebäude hat genau ein einzelne id)# label => Adressstring der vom Server zurückgegeben wird (nicht derjenige, den wir eigeben)
# label => Adressstring der vom Server zurückgegeben wird (nicht derjenige, den wir eigenben)
# Server request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

# Get data in json-format
data = json.loads(r.content)
data

# Take only the first server response, convert to data frame with relevant infos
df = pd.DataFrame.from_dict(list(data.values())[0][0], orient='columns')
df.iloc[[1,4,5,6,11,12],:1]

,attrs
featureId,3149488_0
label,Täliweg 28 <b>8887 Mels</b>
lat,47.042603
lon,9.422916
x,212080.59375
y,750782.125


## Geocoding multiple addresses

### Importing apartment data

In [7]:
# Get current working directory
print(os.getcwd())

# Show all files in the directory
flist = fnmatch.filter(os.listdir('.'), '*.csv')
for i in flist:
    print(i)

# Read the data to a pandas data frame
df = pd.read_csv('apartments_data_prepared.csv', 
                 sep=',', 
                 encoding='utf-8')[['web-scraper-order', 
                                    'address_raw', 
                                    'datetime', 
                                    'rooms', 
                                    'area', 
                                    'luxurious', 
                                    'price_per_m2']][:100] # first 100 apartment adresses

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

C:\Workspacezhaw\data_analytics\Woche 5
apartments_data_geocoded.csv
apartments_data_geocoded_all.csv
apartments_data_prepared.csv
qgis_sys_paths.csv
supermarkets_data_prepared.csv
(100, 7)


,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price_per_m2
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2022-09-07 09:00:00,3.5,122,1,26.07
1,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2022-09-07 09:00:00,2.5,78,0,48.21
2,1662023742-807,"Langfurrenstrasse 5c, 8623 Wetzikon ZH, ZH",2022-09-07 09:00:00,5.5,115,0,24.87
3,1662023804-1290,"Sandbuckweg 5A, 8157 Dielsdorf, ZH",2022-09-07 09:00:00,3.5,74,0,29.26
4,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2022-09-07 09:00:00,5.5,195,1,35.38


### Geocoding multiple apartment addresses using the geoadmin API

In [8]:
# Define base url
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of adresses
geolocation = []
n = 1
for i in df['address_raw'].astype(str): #leere lsite nammens geolocation wird erweitert
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df['address_raw']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        # Set up search parameters - address, origins and type
        parameters = {"searchText": i,
                      "origins": "address",
                      "type": "locations",
                     }

        # Server request
        r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

        # Get data
        data = json.loads(r.content)

        # Take first server response, convert to df with relevant infos
        df_loc = pd.DataFrame.from_dict(list(data.values())[0][0], 
                                        orient='columns')
        geolocation.append(df_loc.iloc[[5,6],0].astype(float))
    
    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        
        

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df['address_raw'])))
df['lat'] = df_loc['lat']
df['lon'] = df_loc['lon']
df.head(5)

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price_per_m2,lat,lon
0,1662023695-433,"Sunnenbergstrasse 15, 8633 Wolfhausen, ZH",2022-09-07 09:00:00,3.5,122,1,26.07,47.255714,8.804976
1,1662023745-820,"Lavaterstr. 63, 8002 Zürich, ZH",2022-09-07 09:00:00,2.5,78,0,48.21,47.361378,8.533339
2,1662023742-807,"Langfurrenstrasse 5c, 8623 Wetzikon ZH, ZH",2022-09-07 09:00:00,5.5,115,0,24.87,47.328632,8.810400
3,1662023804-1290,"Sandbuckweg 5A, 8157 Dielsdorf, ZH",2022-09-07 09:00:00,3.5,74,0,29.26,47.477493,8.456285
4,1662023739-771,"Parkring 59, 8002 Zürich, ZH",2022-09-07 09:00:00,5.5,195,1,35.38,47.366898,8.528817


### Plot addresses on map

In [9]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Add lat/lon of addresses
df_sub = df.dropna()
for i in range(0, len(df_sub)):
    folium.Marker(location=(df_sub.iloc[i]['lat'], 
                            df_sub.iloc[i]['lon']), 
                  popup=df_sub.iloc[i]['address_raw']).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

# Plot map
m

### Save data to file

In [10]:
df.to_csv('apartments_data_geocoded.csv', 
           sep=",", 
           encoding='utf-8',
           index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [11]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2023-01-21 17:21:25
Python Version: 3.9.7
-----------------------------------
